In [ ]:
using System;
using System.Collections;
using System.Collections.Generic;
using System.Linq;
using System.Numerics;

public static class Geometry
{

	/// <summary>
	/// Shrink a polygon with anti-clockwise winding by moving its edges inwards.
	/// Outputs a list of anti-clockwise winded polygons and returns a success flag.
	/// </summary>
	public static bool OffsetPolygonInwards(
		IList<Vector2> polygonPoints, float offset, out List<List<Vector2>> polygons )
	{
		polygons = new List<List<Vector2>>();
		
		polygons = ShrinkEdges(polygonPoints.ToList(), offset);
		
		bool success = true;

		return success;
	}

	public static float Cross2(Vector2 v, Vector2 w)
		=> Vector3.Cross(new Vector3(v, 0), new Vector3(w, 0)).Z;

	public static List<(int, int, int)> Triples(int n)
	{
		var p = Enumerable.Range(0, n).ToList();
		var a = Enumerable.Range(0, n).ToList();
		var b = a.Select(x => (x + 1) % n).ToList();
		var res = new List<(int, int, int)>();
		foreach (int i in p){
			foreach (int j in a){
				var k = (j + 1) % n;
				if (j != (i - 1 + n) % n && k != (i + 1) % n)
					res.Add((i, j, k));
			}
		}
		return res;
	}

	public static List<int> SplitPolyL(int pi, int ai, int n)
    => Enumerable.Range(pi, n).Select(i => i % n).TakeWhile(i => i != ai).Append(ai).ToList();

	public static List<int> SplitPolyR(int pi, int bi, int n)
    => Enumerable.Range(bi, n).Select(i => i % n).TakeWhile(i => i != pi).Append(pi).ToList();
  
  public static List<float> RootsDeg2(float a, float b, float c)
  {
    var d = b * b - 4 * a * c;
    if (d < 0) return new List<float>();
    var e = - b / (2 * a);
    if (d == 0) return new List<float>() { e };
    var x = (float) (e + Math.Sqrt(d / (2 * a)));
    var y = (float) (e - Math.Sqrt(d / (2 * a)));
    return new List<float>() { x, y };
  }

	public static List<float> LineSegCol(Vector2 a0, Vector2 b0, Vector2 va, Vector2 vb)
	{
    const float threshold = 0.001f;
    var ca = Cross2(va, vb);
    var cb = Cross2(a0, vb) - Cross2(b0, va);
    var cc = Cross2(a0, b0);
    var roots = RootsDeg2(ca, cb, cc);
    Func<float, bool> cond2 = t => Vector2.Dot(a0 + va * t, b0 + vb * t) < threshold;
    return roots.Where(t => (t > -threshold) && (cond2(t))).ToList();
	}

	public static List<List<Vector2>> SimpleScale(IEnumerable<Vector2> polyIn, float offset)
		=> new List<List<Vector2>>() { polyIn.Select(x => x * offset).ToList() };
	
	
	
	public static List<List<Vector2>> ShrinkEdges(List<Vector2> p, float t, int iter = 0)
	{
		int debugN = 0;
		int n = p.Count();
		const int maxIter = 10;
		if (iter > maxIter || n < 3) return new List<List<Vector2>>(); 
		iter ++;
		Console.Write(debugN.ToString()); debugN ++; // 0
		var nl = p.Select((x, i) => p[(i - 1 + n) % n]);
		Console.Write(debugN.ToString()); debugN ++; // 1
		var nr = p.Select((x, i) => p[(i + 1) % n]);
		Console.Write(debugN.ToString()); debugN ++; // 2
		var vl = nl.Select((x, i) => Vector2.Normalize(x - p[i])).ToList();
		Console.Write(debugN.ToString()); debugN ++; // 3
		var vr = nr.Select((x, i) => Vector2.Normalize(x - p[i])).ToList();
		Console.Write(debugN.ToString()); debugN ++; // 4
		var cp = vl.Select((x, i) => Cross2(x, vr[i])).ToList();
		Console.Write(debugN.ToString()); debugN ++; // 5
		var vd = vl.Select((x, i) => (x + vr[i]) / cp[i]).ToList();
		Console.Write(debugN.ToString()); debugN ++; // 6
		var triples = Triples(n);
		Console.Write(debugN.ToString()); debugN ++; // 7
		var cols = triples.SelectMany((trpl) => {
			var (i, j, k) = trpl;
			var rs = LineSegCol(
				p[j] - p[i],
				p[k] - p[i],
				vd[j] - vd[i],
				vd[k] - vd[i]);
			return rs.Select(r => (i, j, k, r, (p[i] + vd[i]) * r)).ToList();
		});
		Console.Write(debugN.ToString()); debugN ++; // 8
    Func<float, List<Vector2>> ptt = t => p.Select((v, i) => v + vd[i] * t).ToList();
		Console.Write(debugN.ToString()); debugN ++; // 9
    if (cols.Count() == 0) return new List<List<Vector2>>() { ptt(t) };
    else {
			Console.Write(debugN.ToString()); debugN ++; // 10
      var (fpi, fai, fbi, ft, fcp) = cols.OrderBy(x => x.Item4).First();
			Console.Write(debugN.ToString()); debugN ++; // 11
      if (ft < t) return new List<List<Vector2>>() { ptt(t) };
      else {
				Console.Write(debugN.ToString()); debugN ++; // 12
        var pft = ptt(ft);
				Console.Write(debugN.ToString()); debugN ++; // 13
        var polyL = SplitPolyL(fpi, fai, n).Select(i => pft[i]).ToList();
				Console.Write(debugN.ToString()); debugN ++; // 14
        var polyR = SplitPolyR(fpi, fai, n).Select(i => pft[i]).ToList();
				Console.Write(debugN.ToString()); debugN ++; // 15
        return ShrinkEdges(polyL, t - ft, iter).Concat(ShrinkEdges(polyR, t - ft, iter)).ToList();
      }
    }
	}
}

In [ ]:
var poly1 = (new List<(float, float)> { (0, 0), (1, 1), (1, 0), (0.9f, 0.1f), (0.8f, 0.0f) })
  .Select(x => new Vector2(x.Item1, x.Item2)).ToList();

Geometry.ShrinkEdges(poly1, 0.6f)

0123456789

index,value
0,"[ { <1.4485283, 0.6>: X: 1.4485283, Y: 0.6 }, { <0.39999998, -0.4485283>: X: 0.39999998, Y: -0.4485283 }, { <0.39999998, 1.4485279>: X: 0.39999998, Y: 1.4485279 }, { <0.89999986, 0.94852823>: X: 0.89999986, Y: 0.94852823 }, { <0.55147177, 0.6>: X: 0.55147177, Y: 0.6 } ]"


In [ ]:
(int, int, int, int) x = (1,2,3,4);
var (y,z,v,w) = x;
x.Item4

4

In [ ]:
-1 % 10

-1

In [ ]:
Geometry.Triples(4)

index,Item1,Item2,Item3
0,0,1,2
1,0,2,3
2,1,2,3
3,1,3,0
4,2,0,1
5,2,3,0
6,3,0,1
7,3,1,2


In [ ]:
(new List<int>() {1, 2, 3, 4, 5} ).Select((x, i) => (x, i)).ToList()

index,Item1,Item2
0,1,0
1,2,1
2,3,2
3,4,3
4,5,4


In [ ]:
float x = 1;
Console.Write("eheee");

eheee
